In [168]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
from IPython.display import Image
from qutip import *
import math
import json

In [169]:
wc = 0.5  # cavity frequency
wa = 0.5  # atom frequency
g = 1.0 #cavity coupling
mu_gg = 1.0 #ground state permanent dipole
mu_ee = 0.5 #excited state permanent dipole
mu_eg = 1.5 #transition dipole
N = 2 #number of fock states
lambda_val = 0.01

PF_data = {

    "spin_frequency" : wa,
    "cavity_frequency" : wc,
    "cavity_coupling" : g,
    "ground_state_permanent_dipole" : mu_gg,
    "excited_state_permanent_dipole" : mu_ee,
    "transition_dipole" : mu_eg,
    "number_of_fock_states" : N,
    "use_rabi" : False,
    "lambda_value" : lambda_val,
    "energies" : [],
}

Rabi_data = {

    "spin_frequency" : wa,
    "cavity_frequency" : wc,
    "cavity_coupling" : g,
    "ground_state_permanent_dipole" : mu_gg,
    "excited_state_permanent_dipole" : mu_ee,
    "transition_dipole" : mu_eg,
    "number_of_fock_states" : N,
    "use_rabi" : True,
    "lambda_value" : lambda_val,
    "energies" : [],
}

In [170]:
# intial state
psi0 = tensor(basis(N,0), basis(2,1))    # start with an excited atom

# operators
a  = tensor(destroy(N), qeye(2))
sm = tensor(qeye(N), destroy(2))

In [171]:
#calculations for Hamiltonians
mu = mu_gg * sm * sm.dag() + mu_ee * sm.dag() * sm + mu_eg * (sm + sm.dag())
H_pf = wc * a.dag() * a + wa * sm.dag() * sm - np.sqrt(wc/2) * lambda_val * mu * (a.dag() + a) + 1/2 * (lambda_val * mu) ** 2
H_rabi = wc * a.dag() * a + wa * sm.dag() * sm - np.sqrt(wc/2) * lambda_val * mu * (a.dag() + a)

In [172]:
#collect eigenergies for the calculatied hamiltonians
pf_energies = H_pf.eigenenergies().tolist()
rabi_energies = H_rabi.eigenenergies().tolist()
#pf_energies = str(H_pf.eigenenergies().tolist())
#rabi_energies = str(H_rabi.eigenenergies().tolist())

PF_data["energies"].append(pf_energies)
Rabi_data["energies"].append(rabi_energies)

In [173]:
# write the data to a JSON file
def write_to_json(data, filename):
    with open(filename, 'w') as json_file:
        json.dump(data, json_file, indent=4)
    
def name_file(dictionary):
    if dictionary["use_rabi"]:
        output_filename = "Rabi_simulation_"
    else:
        output_filename = "PF_simulation_"
    output_filename += "spin_freq_" + str(dictionary["spin_frequency"]) + "_"
    output_filename += "cavity_freq_" + str(dictionary["cavity_frequency"]) + "_"
    output_filename += "lambda_value_" + str(dictionary["lambda_value"]) + "_"
    output_filename += "N_fock_states_" + str(dictionary["number_of_fock_states"]) + "_"
    output_filename += "cavity_coupling_" + str(dictionary["cavity_coupling"]) + ".json"
    return output_filename

#name files
pf_name = name_file(PF_data)
rabi_file = name_file(Rabi_data)

#write data to json
write_to_json(PF_data, pf_name)
write_to_json(Rabi_data, rabi_file)